# Tunier


In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-artificial-intelligence.ipynb
%run ./nmm-alpha-beta-pruning.ipynb
%run ./nmm-minimax.ipynb

In [ ]:
from typing import Optional, List
from collections import defaultdict

## Runde


In [ ]:
class Round():
    def __init__(
        self,
        white: ArtificialIntelligence, black: ArtificialIntelligence,
        start_state = s0, start_player = 'w',
        max_turns: int = 250, max_state_replayed: int = 5, max_states_without_mill: int = 30,
        name: str = ""
    ):
        self.white = white
        self.black = black
        
        self.state = start_state
        self.player = start_player
        self.max_turns = max_turns
        self.max_state_replayed = max_state_replayed
        self.max_states_without_mill = max_states_without_mill
        self.name = name
        
        self.log = [start_state]
        self.visited = defaultdict(int)
        self.no_mill_played = 0

In [ ]:
def __repr__(self: Round):
    return f"Round(name='{self.name}', white={type(self.white).__name__}, black={type(self.black).__name__}, max_turns={self.max_turns})"

Round.__repr__ = __repr__
del __repr__

In [ ]:
class RoundResult():
    def __init__(self, winner: str, log: List, reason: str):
        self.winner = winner
        self.log    = log
        self.reason = reason
    
    def __repr__(self):
        return f"RoundResult(winner='{self.winner}', log={len(self.log)}, reason='{self.reason}')"

In [ ]:
def current_ai(self: Round) -> ArtificialIntelligence:
    if self.player == 'w':
        return self.white
    return self.black

Round.current_ai = current_ai
del current_ai

In [ ]:
def check_remis(self: Round) -> Optional[RoundResult]:
    if len(self.log) >= self.max_turns:
        return RoundResult(
            winner = ' ',
            log    = self.log,
            reason = f"Reached max_turns after {self.max_turns} turns"
        )
    
    if self.visited[self.state] >= self.max_state_replayed:
        return RoundResult(
            winner = ' ',
            log    = self.log,
            reason = f"State has been replayed for {self.visited[self.state]} turns"
        )
    
    if self.no_mill_played >= self.max_states_without_mill:
        return RoundResult(
            winner = ' ',
            log    = self.log,
            reason = f"No mill has been played for {self.no_mill_played} turns"
        )
    return None

Round.check_remis = check_remis
del check_remis

In [ ]:
def play(self: Round) -> RoundResult:
    while True:
        if finished(self.state, self.player):
            return RoundResult(
                winner = ('w', ' ', 'b')[utility(self.state, 'b')+1],
                log    = self.log,
                reason = f"A player won the round"
            )
        
        mills_before = findMills(self.state[1], self.player)
        remis = self.check_remis()
        if remis is not None:
            return remis
        
        bestMoves = self.current_ai().bestMoves(self.state, self.player)
        self.state  = bestMoves.choice()
        self.player = opponent(self.player)
        
        self.log.append(bestMoves)
        self.visited[self.state] += 1
        if countNewMills(self.state[1], mills_before, self.player) <= 0:
            self.no_mill_played += 1
        else:
            self.no_mill_played = 0
        
        print(f"{self.name} t={len(self.log): <3} {self.player} found {len(bestMoves.states): >2} states with value={bestMoves.value}")
    
    

Round.play = play
del play

## Wettkampf


In [ ]:
from multiprocessing import cpu_count, Pool

In [ ]:
class Tournament():
    def __init__(self, white: ArtificialIntelligence, black: ArtificialIntelligence, instances: int = None):
        self.white = white
        self.black = black
        
        self.instances = instances
        if self.instances is None:
            self.instances = cpu_count()

In [ ]:
def execute(element):
    idx, rnd = element
    random.seed(idx)
    return rnd.play()

In [ ]:
def play(self: Tournament):
    rounds = [
        Round(
            self.white() if callable(self.white) else self.white,
            self.black() if callable(self.black) else self.black,
            name = f"r={i: <2}"
        )
        for i in range(self.instances)
    ]
    with Pool(self.instances) as pool:
        return pool.map(execute, enumerate(rounds))

Tournament.play = play
del play